In [1]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.7 MB/s eta 0:00:00


In [2]:
import chess.pgn
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [3]:
def board_to_matrix(board: chess.Board, move: chess.Move):
    matrix_board = torch.zeros((6, 8, 8))
    matrix_move = torch.zeros((8, 8))

    for i in range(8):
        for j in range(8):
            piece = board.piece_at(chess.square(i, j))
            if piece is not None:
                piece_type = piece.piece_type
                piece_color = piece.color
                index = piece_type - 1
                if piece_color == chess.WHITE:
                    matrix_board[index, 7-j, i] = 1
                else:
                    matrix_board[index, 7-j, i] = -1
                # print(matrix_board)
    if board.turn == chess.BLACK:
        matrix_board *= -1

    file = chess.square_file(move.from_square)
    rank = chess.square_rank(move.from_square)
    matrix_move[7-rank][file] = 1
    # print(matrix_board)
    return matrix_board.tolist(), matrix_move.tolist()

def main():
    pgn = open("full.pgn")
    cnt = 0
    board_matrix, piece_matrix = [], []

    while True:
        game = chess.pgn.read_game(pgn)
        if game is None:
            break
        cnt+=1
        if cnt % 100 == 0:
            print(cnt)

        board = game.board()
        for move in game.mainline_moves():
            if board.turn == chess.WHITE:
                matrix_board, matrix_move = board_to_matrix(board, move)
                board_matrix.append(matrix_board)
                piece_matrix.append(matrix_move)
            board.push(move)


    X = torch.tensor(board_matrix)
    y = torch.tensor(piece_matrix)
    torch.save(X, "X.pt")
    torch.save(y, "y.pt")

main()

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [ ]:
def test():
    X = torch.load("X.pt")
    y = torch.load("y.pt")
    assert len(X) == len(y)

    ratio = 0.8
    idx = int(X.size(0)*ratio)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    X_train, X_test = X[:idx].to(device), X[idx:].to(device)
    Y_train, Y_test = y[:idx].to(device), y[idx:].to(device)
    print(torch.cuda.is_available())
    print(device)

    model = nn.Sequential(
        nn.Flatten(),
        nn.Linear(8*8*6, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 16),
        nn.ReLU(),
        nn.Linear(16, 8*8),
    ).to(device)

    X_train.to(device)
    X_test.to(device)
    Y_train.to(device)
    Y_test.to(device)

    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    num_epochs = 3000
    eps = []
    train_loss = []
    valid_loss = []

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        for inputs, labels in zip(X_train, Y_train):
          inputs = inputs.view(-1, 8*8*6)
          labels = labels.view(-1, 8*8)
          output = model(inputs)
          loss = criterion(output, labels)
          loss.backward()
          optimizer.step()
        model.eval()

        if (epoch+1) % 10 == 0:
          total_loss = 0
          with torch.no_grad():
            for val_inputs, val_labels in zip(X_test, Y_test):
                val_inputs = val_inputs.view(-1, 8*8*6)
                val_labels = val_labels.view(-1, 8*8)
                val_outputs = model(val_inputs)
                val_loss = criterion(val_outputs, val_labels)
                total_loss += val_loss.item() * val_inputs.size(0)
                num_samples += val_inputs.size(0)
            avg_val_loss = total_loss / num_samples

            print(f'Epoch [{epoch+1}/{num_epochs}], train_loss: {loss.item():.4f}, val_loss: {avg_val_loss}')
            eps.append(epoch)
            train_loss.append(loss.item())
            valid_loss.append(avg_val_loss)

    plt.plot(eps, train_loss, label="train loss")
    plt.plot(eps, valid_loss, label="valid loss")
    plt.xlabel("eps")
    plt.ylabel("loss")
    plt.legend()
    plt.show()


if __name__ == "__main__":
    # main()
    test()


True
cuda
